In [1]:
%config Completer.use_jedi = False #for auto complete code 

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import skew
#%matplotlib inline
plt.style.use('ggplot')

#funcion parachekar si los eneros estan completos
def mescompleto(x,ano):
    a=x[x.index.year==ano].isnull().sum()==0
    return a
#retorna años con los meses completos
def anosconmesescompletos(x):
    ano=range(1981,2017)
    idx=[]
    for i in range(len(ano)):
        if mescompleto(x,ano[i]):
            idx.append(ano[i])
    return idx

#retorna la serie con los meses completos para luego calcular los estadisticos
def seriefinal(x):

    idx=anosconmesescompletos(x)
    lista=[]
    for i in range(len(idx)):
        lista.append(x[x.index.year==idx[i]])
    
    if len(lista)>15:#mayor a 15 ano
        return pd.concat(lista)#remodificar esta funcion
    else:
        return []

def proportiondry(x):
    def mayorque(y):
        return y>0.1
    drydays=len(x)-np.sum(x.apply(mayorque))
    return float(drydays)/len(x)

def autocovariance(Xi, k=1):
    N = np.size(Xi)
    Xs = np.average(Xi)
    autoCov = 0
    for i in np.arange(0, N-k):
        autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
    return (1.0/(N-1))*autoCov

def estadisticosprec(x):
    result=[]
    a=x.dropna()
    #si se tiene menor a 10 años no contar
    if len(a)==0:
        return [None]
    result.append(np.mean(a))
    result.append(np.var(a))
    result.append(autocovariance(a))
    result.append(proportiondry(a))
    return result

#crea una tabla de los estadisticos extraidos una vez usado la fucion estadisticosprec
def crearTablaEst(x,names=['x','y','mean1h','var1h','autocov1h','dryperiod1h']):
    
    val=x.transpose()
    val.reset_index(drop=True, inplace=True)
    pos=posiciones#posiciones.loc[final_station,:]
    pos.reset_index(drop=True, inplace=True)
    df=pd.concat([pos, pd.DataFrame(np.matrix(val))], axis=1)
    
    df.columns=names
    
    return df

def factores(a,b):
    if a==0.0 or b==0.0:
        return a
    else:
        return float(a)/b

#  Correcion de los datos IMERG a Nivel Nacional

In [4]:
#Las 372 estaciones iniciales Luego tienen que ser filtradas :v
datos=pd.read_csv('D:/Proyectos_GitHub/TrmmCorrection/data/IMERG_NATIONAL_FEB.csv')
rng = pd.date_range('2002-01-01 00:00:00','2016-12-31 21:00:00', freq='0.5H')
fechasTR=rng[np.in1d(rng.month, 2)]#filtra las fechas para que solo sean de febrero
posiciones=datos.iloc[:,0:2]
posiciones.columns=['x','y']
TRMM=datos.iloc[:, 2::].transpose()
TRMM=TRMM.set_index(fechasTR)
TRMM.head()

,0,1,2,3,4,5,6,7,8,9,...,362,363,364,365,366,367,368,369,370,371
2002-02-01 00:00:00,0.0,0.0,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,1.435950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2002-02-01 00:30:00,0.0,0.0,0.029940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007839,0.018295,0.096015,1.417870,0.000149,0.018305,0.000781,0.001588,0.017935,0.003127
2002-02-01 01:00:00,0.0,0.0,0.020893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.019047,0.059329,0.233285,0.939371,0.000363,0.044474,0.001899,0.003858,0.043577,0.007597
2002-02-01 01:30:00,0.0,0.0,0.014945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.160786,0.998990,0.699576,0.951648,0.144467,0.286831,0.018335,0.010826,0.107958,0.018820
2002-02-01 02:00:00,0.0,0.0,0.009708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.761963,0.607715,1.479626,0.474087,0.002271,0.278466,0.011888,0.024158,0.272848,0.047565


In [8]:
rng = pd.date_range('1981-01-01','2016-12-31', freq='d')
fechasOB=rng[np.in1d(rng.month, 2)]#
obs24=pd.read_csv('D:/Proyectos_GitHub/TrmmCorrection/data/RAIN_GAUGE.csv')
prec=obs24.iloc[:,3:13152].transpose()
prec=prec.set_index(rng)

In [9]:
#obtengo todos los febreros para las estaciones a nivel nacional
precFeb=prec.loc[fechasOB]
TrmmDiario=TRMM.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [10]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng = pd.date_range('2002-02-01','2016-12-28', freq='d')
fechasD=rng[np.in1d(rng.month, 2)]#
TrmmDiario=TrmmDiario.loc[fechasD]
obsDiario=prec.loc[fechasD]

#  Trabajando con meses completos

In [11]:
idx=[]
n=0.8*480
for x in obsDiario.columns:
    if obsDiario[x].count()> n:
        idx.append(x)
        
#idx=list(obsDiario.isnull().sum()==0) #pixeles con meses completos
posiciones=posiciones.iloc[idx,:] #corrdenadas de los meses completos
TrmmDiario=TrmmDiario.iloc[:,idx]
obsDiario=obsDiario.iloc[:,idx]
TRMM=TRMM.iloc[:,idx]

In [12]:
BIASM=[]
for i in range(0,TrmmDiario.shape[0]):#filas
    BIAS=[]
    for j in range(0,TrmmDiario.shape[1]):#columnas
        BIAS.append(factores(obsDiario.iloc[i,j],TrmmDiario.iloc[i,j]))
    BIASM.append(BIAS)

    correcionM=[]
bugM=[]
k=0 #lleva la contabilidad de las fechas diarias
for i in range(0,TRMM.shape[0]):
    correcion=[]
    bug=[]
    for j in range(0,TRMM.shape[1]):
        if fechasTR[i].date()==fechasD[k].date():
            
            bug.append(BIASM[k][j])
            correcion.append(TRMM.iloc[i,j]*BIASM[k][j])
            k=k
        else:
            k=k+1
            correcion.append(TRMM.iloc[i,j]*BIASM[k][j])
    
    bugM.append(bug)
    correcionM.append(correcion)

In [13]:
DATOSTRMMCORRE=pd.DataFrame(correcionM)
DATOSTRMMCORRE=DATOSTRMMCORRE.set_index(fechasTR)

In [16]:
ESTTRMMD=crearTablaEst(DATOSTRMMCORRE.groupby(pd.Grouper(freq='24h')).agg(np.sum).loc[fechasD].apply(estadisticosprec, axis=0),names=['x','y','mean24h','var24h','autocov24h','dryperiod24h'])
ESTTRMMD

,x,y,mean24h,var24h,autocov24h,dryperiod24h
0,-80.53361,-3.633611,2.802618,79.380640,24.907011,0.707547
1,-80.45750,-3.813333,4.825442,224.617751,22.954293,0.641509
2,-80.45722,-3.508333,3.573585,178.839632,31.480008,0.747642
3,-80.32194,-3.440556,3.249555,94.749908,18.459478,0.714623
4,-80.66083,-3.948889,2.855991,69.863059,10.236787,0.768868
...,...,...,...,...,...,...
226,-70.05028,-17.116944,3.748255,35.779718,13.862996,0.466981
227,-69.99972,-17.559167,2.941392,25.990043,11.686709,0.533019
228,-69.81306,-17.237222,4.042500,37.514138,15.959935,0.490566
229,-69.77944,-17.525000,3.271934,26.754896,7.182733,0.464623


In [17]:
EST24=obsDiario.apply(estadisticosprec, axis=0)
EST24_table=crearTablaEst(EST24,names=['x','y','mean24h','var24h','autocov24h','dryperiod24h'])
EST24_table

,x,y,mean24h,var24h,autocov24h,dryperiod24h
0,-80.53361,-3.633611,3.476313,87.514540,24.526690,0.583333
1,-80.45750,-3.813333,5.766008,232.449585,26.934913,0.471698
2,-80.45722,-3.508333,3.913239,181.014056,31.857021,0.692671
3,-80.32194,-3.440556,4.066851,116.696489,20.215274,0.611374
4,-80.66083,-3.948889,3.906110,82.579682,12.834200,0.639618
...,...,...,...,...,...,...
226,-70.05028,-17.116944,4.110153,37.298945,13.545842,0.404092
227,-69.99972,-17.559167,3.064679,26.103121,11.398436,0.501188
228,-69.81306,-17.237222,4.234245,38.366804,17.035430,0.464623
229,-69.77944,-17.525000,3.287500,26.741141,7.116114,0.459906


# Probando Varianza de 1,3,6, 12, y 18 horas Febrero

In [18]:
DATA1=DATOSTRMMCORRE.groupby(pd.Grouper(freq='1h')).agg(np.sum)
EST1=DATA1.loc[np.in1d(DATA1.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
DATA3=DATOSTRMMCORRE.groupby(pd.Grouper(freq='3h')).agg(np.sum)
EST3=DATA3.loc[np.in1d(DATA3.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
DATA6=DATOSTRMMCORRE.groupby(pd.Grouper(freq='6h')).agg(np.sum)
EST6=DATA6.loc[np.in1d(DATA6.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
DATA12=DATOSTRMMCORRE.groupby(pd.Grouper(freq='12h')).agg(np.sum)
EST12=DATA12.loc[np.in1d(DATA12.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
DATA18=DATOSTRMMCORRE.groupby(pd.Grouper(freq='18h')).agg(np.sum)
EST18=DATA18.loc[np.in1d(DATA18.index.month, 2)].dropna().apply(estadisticosprec, axis=0)

In [19]:
names=['stattion','x','y']

agregados=[24,1,3,6,12,18]
ESTS=[EST24,EST1,EST3,EST6,EST12,EST18]
result=[]
for i,stat in enumerate(ESTS):
    names=['x','y']
    names=names+['mean{}'.format(agregados[i]),'var{}'.format(agregados[i]),'autocov{}'.format(agregados[i]),'dryperiod{}'.format(agregados[i])]
    result.append(crearTablaEst(stat,names))

In [20]:
df=pd.concat(result,axis=1)
df=df.loc[:,~df.columns.duplicated()]

In [21]:
df

,x,y,mean24,var24,autocov24,dryperiod24,mean1,var1,autocov1,dryperiod1,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,3.476313,87.514540,24.526690,0.583333,0.116776,0.878095,0.416378,0.930228,...,2.866572,0.854953,1.401309,32.716619,5.657919,0.792453,2.107357,55.087104,12.876025,0.731794
1,-80.45750,-3.813333,5.766008,232.449585,26.934913,0.471698,0.201060,2.479182,1.291029,0.919517,...,7.099055,0.824292,2.412721,100.198220,7.799999,0.753538,3.610568,153.494605,15.601931,0.667851
2,-80.45722,-3.508333,3.913239,181.014056,31.857021,0.692671,0.148899,2.865244,0.940596,0.933864,...,4.834914,0.855542,1.786792,79.608987,7.206378,0.801887,2.681630,121.457961,25.576878,0.751332
3,-80.32194,-3.440556,4.066851,116.696489,20.215274,0.611374,0.135398,1.271232,0.573853,0.928950,...,3.163217,0.848467,1.624777,38.651189,6.778821,0.781840,2.440467,66.892928,7.982899,0.730018
4,-80.66083,-3.948889,3.906110,82.579682,12.834200,0.639618,0.119000,0.980255,0.457506,0.941726,...,2.253381,0.879127,1.427995,32.628546,1.583828,0.829009,2.104512,49.933071,5.176535,0.779751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-70.05028,-17.116944,4.110153,37.298945,13.545842,0.404092,0.156177,0.410294,0.202016,0.826847,...,1.110869,0.619693,1.874127,14.047952,3.063857,0.520047,2.808294,23.932576,8.366559,0.463588
227,-69.99972,-17.559167,3.064679,26.103121,11.398436,0.501188,0.122558,0.373290,0.174991,0.894752,...,0.660003,0.748821,1.470696,11.089713,2.123675,0.637972,2.191121,18.027902,6.477140,0.541741
228,-69.81306,-17.237222,4.234245,38.366804,17.035430,0.464623,0.168437,0.545656,0.265123,0.827535,...,1.273952,0.650943,2.021250,16.427154,1.846077,0.557783,3.023391,26.989317,7.932189,0.472469
229,-69.77944,-17.525000,3.287500,26.741141,7.116114,0.459906,0.136331,0.280678,0.140838,0.812991,...,0.792766,0.602594,1.635967,10.905042,1.737815,0.516509,2.397909,17.179736,4.825930,0.447602
